In [1]:
import pandas as pd
import numpy as np
import math
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"

MoM FFBL Weekly Rankings Replacement 

In [ ]:
"""
MoM FFBL Weekly Rankings Replacement 2pt system
"""
matchups_query = """SELECT * FROM dev.regseasonmatchups"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)

matchups_a = matchups.copy()
matchups_b = matchups.copy()

matchups_b_cols = list(matchups_b.columns)

rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"

matchups_b.rename(columns=rename_columns, inplace=True)

matchups = pd.concat([matchups_a, matchups_b])

matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)

matchups.reset_index(drop=True, inplace=True)

matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
    ],
    axis=1,
    inplace=True,
)

matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)

matchups["2pt_system"] = matchups.groupby(["week", "game_id"])["team_a_points"].rank(
    "first", ascending=False
)

matchups["2pt_system"] = np.where(matchups["2pt_system"] <= 5, 1, 0)

matchups = matchups[
    [
        "week",
        "team_a_team_key",
        "team_a_points",
        "team_a_projected_points",
        "win_loss",
        "2pt_system",
        "game_id",
    ]
]

weeks = list(matchups["week"].unique())

regular_season = matchups[matchups["week"] == 1]

for wk in weeks[1:]:
    reg_season_week = matchups[matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )

reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_system" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)
regular_season.insert(2, "team_a_points_total", regular_season[team_pts].sum(axis=1))
regular_season.insert(
    3, "team_a_project_points_total", regular_season[team_pro_pts].sum(axis=1)
)
regular_season.insert(
    4, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    5, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(2, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(6, "week_1", week_1)
regular_season.insert(
    2,
    "w_l_rank",
    regular_season.groupby(["game_id"])["wins"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    2,
    "2pt_rank",
    regular_season.groupby(["game_id"])["2pt_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    5,
    "pts_rank",
    regular_season.groupby(["game_id"])["team_a_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    7,
    "projected_pts_rank",
    regular_season.groupby(["game_id"])["team_a_project_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)

df_2pt = regular_season[regular_season["game_id"] >= 390]
df_2pt.insert(0, "tuple", df_2pt[["2pt_rank", "pts_rank"]].apply(tuple, axis=1))

df_h2h = regular_season[regular_season["game_id"] < 390]
df_h2h.insert(0, "tuple", df_h2h[["w_l_rank", "pts_rank"]].apply(tuple, axis=1))

regular_season = pd.concat([df_2pt, df_h2h])

regular_season.insert(
    0,
    "reg_season_rank",
    regular_season.groupby(["game_id"])["tuple"]
    .rank(method="min", ascending=True)
    .astype(int),
)
regular_season = regular_season.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)

regular_season["name"] = regular_season["name"].fillna(
    regular_season["team_a_team_key"]
)
regular_season["nickname"] = regular_season["nickname"].fillna(
    regular_season["team_a_team_key"]
)

team_name = regular_season["name"]
team_mang = regular_season["nickname"]
game_id = regular_season["game_id_x"]
regular_season.drop(
    ["name", "nickname", "tuple", "game_id_x", "game_id_y"],
    axis=1,
    inplace=True,
)
regular_season.insert(1, "manager", team_mang)
regular_season.insert(2, "team_name", team_name)
regular_season.insert(0, "game_id", game_id)
regular_season.sort_values(
    ["reg_season_rank", "2pt_total", "team_a_points_total"],
    ascending=[True, False, False],
    inplace=True,
)
regular_season.drop("team_key", axis=1, inplace=True)

rename_columns_2 = {}
drop_columns = []
for col in regular_season.columns:
    if "team_a_" in col:
        rename_columns_2[col] = col[len("team_a_") :]
    if "game_id_" in col:
        drop_columns.append(col)

regular_season.rename(columns=rename_columns_2, inplace=True)
regular_season.drop(drop_columns, axis=1, inplace=True)

rename_columns_3 = {}
for col in regular_season.columns[15:]:
    if not str(col)[-1].isnumeric():
        rename_columns_3[col] = f"{col}_1"

regular_season.rename(columns=rename_columns_3, inplace=True)


DatabaseCursor(PATH, options=OPTION_PROD).copy_table_to_postgres_new(
    df=regular_season, table="regseasonweeklyrankings", first_time="yes"
)

reg_season_rankings = regular_season[
    [
        "game_id",
        "reg_season_rank",
        "manager",
        "team_name",
        "2pt_total",
        "2pt_rank",
        "points_total",
        "pts_rank",
        "project_points_total",
        "projected_pts_rank",
        "wins",
        "losses",
        "team_key",
    ]
]

DatabaseCursor(PATH, options=OPTION_PROD).copy_table_to_postgres_new(
    df=reg_season_rankings, table="regseasonrankings", first_time="yes"
)

#MOM FFBL Playoff Bracket

In [2]:
endofseason_rankings_query = """SELECT game_id, reg_season_rank, manager, team_name, team_key FROM prod.regseasonrankings where game_id = 273"""
eos_rankings = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres(
    endofseason_rankings_query
)


league_settings_query = (
    """SELECT playoff_start_week, num_playoff_teams, game_id from dev.leaguesettings where game_id = 273"""
)
league_settings = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    league_settings_query
)


team_points_weekly_query = """SELECT * from dev.weeklyteampoints where game_id = 273 and week >= 12"""
team_points_weekly = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    team_points_weekly_query
)


teams_query = """SELECT team_key, name, nickname, game_id, league_id, team_id FROM dev.leagueteams where game_id = 273"""
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)


# team_points_weekly = team_points_weekly.merge(teams, how='outer', left_on=['game_id', 'league_id', 'team_id'], right_on=['game_id', 'league_id', 'team_id'])
# team_key = team_points_weekly['game_id'].astype(str)+"."+team_points_weekly['league_id'].astype(str)+".t."+team_points_weekly['team_id'].astype(str)
# team_points_weekly['team_key'].fillna(team_key, inplace=True)

Successfully pulled: SELECT game_id, reg_season_rank, manager, team_name, team_key FROM prod.regseasonrankings where game_id = 273
Successfully pulled: SELECT playoff_start_week, num_playoff_teams, game_id from dev.leaguesettings where game_id = 273
Successfully pulled: SELECT * from dev.weeklyteampoints where game_id = 273 and week >= 12
Successfully pulled: SELECT team_key, name, nickname, game_id, league_id, team_id FROM dev.leagueteams where game_id = 273


In [3]:
team_points_weekly

,final_points,week,projected_points,team_id,game_id,league_id
0,112.84,12,89.86,1,273,777818
1,74.64,12,96.24,11,273,777818
2,67.00,12,79.85,3,273,777818
3,104.28,12,87.15,4,273,777818
4,88.76,12,96.08,5,273,777818
5,112.14,12,103.52,6,273,777818
6,79.82,12,98.40,7,273,777818
7,86.24,13,82.95,1,273,777818
8,127.92,13,103.66,11,273,777818
9,75.98,13,88.47,3,273,777818


In [74]:
def playoff_winner(team1, team2, round, league_settings, points_df):

    playoff_week = league_settings["playoff_start_week"].values[0] + (round - 1)
    team1_score = points_df["final_points"][(points_df['team_key'] == team1) & (points_df['week'] == playoff_week)].values[0]
    team2_score = points_df["final_points"][(points_df['team_key'] == team2) & (points_df['week'] == playoff_week)].values[0]

    if team1_score > team2_score:
        winner = team1
        loser = team2
    elif team1_score < team2_score:
        winner = team2
        loser = team1
    else:
        winner = np.nan
        loser = np.nan

    return winner, loser

def games_round(games, round, league_settings, points_df):
    winners = []
    losers = []
    for team1, team2 in games:
        winning_team, losing_team = playoff_winner(team1, team2, round, league_settings, points_df)
        winners.append(winning_team)
        losers.append(losing_team)

    return winners, losers

def plan_games(teams):
    return zip(teams[::2], teams[1::2])

def first_place(eos_rankings, league_settings, points_df):

    if len(list(eos_rankings['game_id'].unique())) > 1:
        print("Error, please only include 1 season into the rankings for bracket build.")

    else:
        game_id = eos_rankings['game_id'].values[0]
        rankings = eos_rankings.sort_values("reg_season_rank")
        first_place_teams = list(
            rankings["team_key"][(rankings["reg_season_rank"] <= league_settings['num_playoff_teams'][league_settings['game_id'] == game_id].values[0])]
        )
        second_place_teams = []
        third_place_teams = []
        fourth_place_teams = []
        round = 0
        while len(first_place_teams) > 1:
            round += 1
            print(f"Round {round}: playoffs: {first_place_teams}")
            if round > 1:
                print(f"Round {round}: playoff consolation: {second_place_teams}")
            playoff_games = plan_games(first_place_teams)
            consolation_games = plan_games(second_place_teams)
            first_place_teams, second_place_teams = games_round(playoff_games, round, league_settings, points_df)
            third_place_teams, fourth_place_teams = games_round(consolation_games, round, league_settings, points_df)

        
        champion = first_place_teams[0]  # only one left
        second_place = second_place_teams[0]
        third_place = third_place_teams[0]
        fourth_place = fourth_place_teams[0]
        print(f"Champion is {champion}")
        print(f"Second Place is {second_place}")
        print(f"Third Place is {third_place}")
        print(f"Fourth Place is {fourth_place}")

first_place(eos_rankings=eos_rankings, league_settings=league_settings, points_df=team_points_weekly)

Round 1: playoffs: ['273.l.777818.t.7', '273.l.777818.t.10', '273.l.777818.t.2', '273.l.777818.t.6', '273.l.777818.t.9', '273.l.777818.t.4', '273.l.777818.t.12', '273.l.777818.t.1']


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams"""
players_query = """SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist"""
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
draft_analysis = draft.merge(
    teams,
    how="left",
    left_on=["team_key", "game_id", "league_id"],
    right_on=["team_key", "game_id", "league_id"],
    suffixes=("_draft", "_teams"),
)
draft_analysis = draft_analysis.merge(
    players,
    how="left",
    left_on=["player_key", "game_id", "league_id"],
    right_on=["player_key", "game_id", "league_id"],
    suffixes=("", "_players"),
)
draft_analysis = draft_analysis[~draft_analysis["name"].isna()]
draft_analysis["clinched_playoffs"].fillna(0, inplace=True)
draft_analysis["team_standings.playoff_seed"].fillna(0, inplace=True)
encoded_grades = {
    "A+": 4,
    "A": 4,
    "A-": 3.7,
    "B+": 3.3,
    "B": 3.0,
    "B-": 2.7,
    "C+": 2.3,
    "C": 2.0,
    "C-": 1.7,
    "D+": 1.3,
    "D": 1.0,
    "na": np.nan,
}
draft_analysis["draft_gpa"] = draft_analysis["draft_grade"].replace(encoded_grades)
draft_analysis["avg_draft_gpa_rank"] = draft_analysis.groupby(["team_standings.rank"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_nickname"] = draft_analysis.groupby(["nickname"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_clinched_playoffs"] = draft_analysis.groupby(
    ["clinched_playoffs"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_playoffseed"] = draft_analysis.groupby(
    ["team_standings.playoff_seed"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_draft_pos"] = draft_analysis.groupby(["pick"])[
    "draft_gpa"
].transform("mean")

MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft_query = """SELECT * FROM dev.draftresults"""
players_query = """SELECT * FROM dev.playerlist"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_a_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("", "_team_a"),
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_b_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("_team_a", "_team_b"),
)

matchups[
    (matchups["is_playoffs"] == 1) & (matchups["is_consolation"] == 0)
].sort_values(
    ["week_start", "is_consolation", "winner_team_key"], ascending=[False, True, True]
)

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""